In [3]:
# Code adapted from Udacity's Google Deep Learning tutorial:
# https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/udacity

# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.

from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
import sklearn
from IPython.display import display, Image
from scipy import ndimage
from six.moves.urllib.request import urlretrieve
import pickle
import tensorflow as tf
import sklearn.model_selection
%matplotlib inline

In [4]:
# Download and extract the CIFAR-10 dataset.

pixel_depth = 255.0
url = 'https://www.cs.toronto.edu/~kriz/'
last_percent_reported = None
data_root = '.' # Change me to store data elsewhere
num_labels = 10

def download_progress_hook(count, blockSize, totalSize):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 5% change in download progress.
  """
  global last_percent_reported
  percent = int(count * blockSize * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent
        
def maybe_download(filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  dest_filename = os.path.join(data_root, filename)
  if force or not os.path.exists(dest_filename):
    print('Attempting to download:', filename) 
    filename, _ = urlretrieve(url + filename, dest_filename, reporthook=download_progress_hook)
    print('\nDownload Complete!')
  statinfo = os.stat(dest_filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', dest_filename)
  else:
    raise Exception(
      'Failed to verify ' + dest_filename + '. Can you get to it with a browser?')
  return dest_filename

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall(data_root)
    tar.close()
    print("And... we're done!")
  return os.listdir(os.path.join(data_root, 'cifar-10-batches-py'))
    
def unpickle(file):
    print ("Unpickling %s" % file)
    with open(file, 'rb') as fo:
        return pickle.load(fo, encoding='latin1')

data_root = os.path.join(data_root, 'cifar-10-batches-py')
train_filename = maybe_download('cifar-10-python.tar.gz', 170498071)
train_files = maybe_extract(train_filename)

def shuffle_data(data, labels):
    data, _, labels, _ = sklearn.model_selection.train_test_split(
        data, labels, test_size=0.0, random_state=42
    )
    return data, labels

def load_data(train_batches):
    data = []
    labels = []
    for data_batch_i in train_batches:
        d = unpickle(
            os.path.join(data_root, data_batch_i)
        )
        data.append(d['data'])
        labels.append(np.array(d['labels']))
    # Merge training batches on their first dimension
    data = np.concatenate(data)
    labels = np.concatenate(labels)
    data = data.astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    length = len(labels)
    data, labels = shuffle_data(data, labels)
    data = (data - (pixel_depth / 2)) / pixel_depth
    return data.reshape(length, 3, 32, 32), labels

train_dataset, train_labels = load_data(
    ["data_batch_{}".format(i) for i in range(1, 6)]
)

test_dataset, test_labels = load_data(["test_batch"])

print("INFO: each dataset's element are of shape 3*32*32:")
print('"print(test_dataset.shape)" --> "{}"\n'.format(test_dataset.shape))
print("From the Caffe documentation: ")
print("The conventional blob dimensions for batches of image data "
      "are number N x channel K x height H x width W.\n")


Found and verified ./cifar-10-batches-py/cifar-10-python.tar.gz
Extracting data for ./cifar-10-batches-py/cifar-10-python. This may take a while. Please wait.
And... we're done!
Unpickling ./cifar-10-batches-py/data_batch_1
Unpickling ./cifar-10-batches-py/data_batch_2
Unpickling ./cifar-10-batches-py/data_batch_3
Unpickling ./cifar-10-batches-py/data_batch_4
Unpickling ./cifar-10-batches-py/data_batch_5
Unpickling ./cifar-10-batches-py/test_batch
INFO: each dataset's element are of shape 3*32*32:
"print(test_dataset.shape)" --> "(10000, 3, 32, 32)"

From the Caffe documentation: 
The conventional blob dimensions for batches of image data are number N x channel K x height H x width W.



In [25]:
# Define the neural network

image_size = 32
num_channels = 3
num_labels = 10
batch_size = 256
patch_size = 5
depth = 64
num_hidden = 128
logs_path = "/tmp/tf-logs"

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

graph = tf.Graph()

with graph.as_default():
  writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels), name="train_dataset")
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels), name="train_labels")
  tf_test_dataset = tf.constant(test_dataset.transpose(0, 2, 3, 1))
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME', name='layer1-conv')
    conv = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME', name='layer1-maxpool')
    hidden = tf.nn.relu(conv + layer1_biases, name='layer1-relu')
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME', name='layer2-conv')
    conv = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME', name='layer2-maxpool')
    hidden = tf.nn.relu(conv + layer2_biases, name='layer2-relu')
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [27]:
# Evaluate the model.
# Try playing with the number of iterations and batch size, and network parameters.

num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_data = batch_data.transpose(0, 2, 3, 1)
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
  tp = test_prediction.eval()
  images = test_dataset.transpose(0, 2, 3, 1)
  print('Test accuracy: %.1f%%' % accuracy(tp, test_labels))

Initialized
Minibatch loss at step 0: 6.068390
Minibatch accuracy: 9.4%
Minibatch loss at step 50: 2.066794
Minibatch accuracy: 28.9%
Minibatch loss at step 100: 1.801312
Minibatch accuracy: 39.1%
Minibatch loss at step 150: 1.749214
Minibatch accuracy: 41.0%
Minibatch loss at step 200: 1.582935
Minibatch accuracy: 41.0%
Minibatch loss at step 250: 1.636399
Minibatch accuracy: 45.3%
Minibatch loss at step 300: 1.544091
Minibatch accuracy: 46.1%
Minibatch loss at step 350: 1.480037
Minibatch accuracy: 48.4%
Minibatch loss at step 400: 1.669302
Minibatch accuracy: 40.6%
Minibatch loss at step 450: 1.379397
Minibatch accuracy: 50.4%
Minibatch loss at step 500: 1.522869
Minibatch accuracy: 46.1%
Minibatch loss at step 550: 1.378336
Minibatch accuracy: 52.0%
Minibatch loss at step 600: 1.457023
Minibatch accuracy: 46.9%
Minibatch loss at step 650: 1.430475
Minibatch accuracy: 48.4%
Minibatch loss at step 700: 1.457910
Minibatch accuracy: 50.0%
Minibatch loss at step 750: 1.285339
Minibatch 